<a href="https://colab.research.google.com/github/MarijanMarkovic/Numerical-modelling-of-differential-equations/blob/main/Navier_Stokes_vj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install -y --no-install-recommends fenics
!rm -rf

In [ ]:
from fenics import*
from matplotlib import pyplot as plt

def solve_NS(n, dt):
  #1 Mesh Gen
  mesh=UnitSquareMesh(n, n, 'crossed')
  
  #2 Finite Element Space and Bc
  degree=1
  V=VectorElement('CG', mesh.ufl_cell(), degree+1)
  Q=FiniteElement('CG', mesh.ufl_cell(), degree)
  
  X=FunctionSpace(mesh, MixedElement([V, Q]))
  
  def boundary(x, on_boundary):
    return on_boundary
  
  def origin(x, on_boundary):
    return near(x[1], 0) and near(x[0], 0)
  
  u_exact=Expression(('-cos(x[0]) * sin(x[1]) * sin(2 * t)'),
                      'sin(x[0]) * cos(x[1]) * sin(2 * t)',
                      degree=degree+1, t=0.0)
  p_exact=Expression('-(cos(2 * x[0]) + cos(2 * x[1]) * sin(2 * t) * sin(2 * t))/4',
                     degree=degree+1, t=0.0)
  
  bc=[
      DirichletBC(X.sub(0), u_exact, boundary),
      DirichletBC(X.sub(1), p_exact, origin, 'pointwise')
  ]
  #3 Problem Definition
  u, p=TrialFunctions(X)
  p, q=TestFunctions(X)
  
  
  Re=Constant(1)
  f_exact=Expression(('-2 * cos(x[0]) * sin(x[1]) * (cos(2 * t) + sin(2 * t)/Re)',
                       '2 * sin(x[0]) * cos(x[1]) * (cos(2 * t) + sin(2 * t)/Re)'),
                      degree=degree+1, t=0.0, Re=Re)
  x_old=Function(X)
  u_old, p_old=x_old.split()
  
  a = (dot(u, v) / Constant(dt) + inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
  L = (dot(u_old, v) / Constant(dt) - dot(grad(u_old) * u_old, v) + dot(f, v)) * dx
  
  #a = (dot(u, v) / Constant(dt) + dot(grad(u) * u_old, v) + inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
  #L = (dot(u_old, v) / Constant(dt) + dot(f, v)) * dx
  
  x=Function(X)
  x_exact = Expression(('u[0]', 'u[1]', 'p'),
                     degree=degree+1, u=u_exact, p=p_exact)
  x.interpolate(x_exact)
  x_old.assign(x)

  solve(a==L, x, bc)
  
  u, p=x.split()
  return u,p

t=0.0
dt=0.5
n=[5]

while t<8:
  for i in n:
    t += dt
  
    u, p=solve_NS(i, dt)
    f_exact.t=t
    u_exact.t=t
    p_exact.t=t

In [ ]:
from fenics import *


# 1. mesh generation
def solve_NS(n, dt):
  mesh = UnitSquareMesh(nx, ny, 'crossed')

  # 2. finite element space
  degree = 1
  V = VectorElement('CG', mesh.ufl_cell(), degree+1)
  Q = FiniteElement('CG', mesh.ufl_cell(), degree)

  X = FunctionSpace(mesh, MixedElement([V, Q]))

  u_exact = Expression((
    '- cos(x[0]) * sin(x[1]) * sin(2 * t)',
    'sin(x[0]) * cos(x[1]) * sin(2 * t)'
  ), degree=degree+1, t=0.0)
  p_exact = Expression(
    '-0.25 * (cos(2*x[0]) + cos(2*x[1])) * sin(2 * t) * sin(2 * t)',
    degree=degree+1, t=0.0)

  def boundary(x, on_boundary):
    return on_boundary

  def origin(x, on_boundary):
    return near(x[0], 0) and near(x[1], 0)

bc = [
    DirichletBC(X.sub(0), u_exact, boundary),
    DirichletBC(X.sub(1), p_exact, origin, 'pointwise')
]

# 3. problem definition
u, p = TrialFunctions(X)
v, q = TestFunctions(X)

x_old = Function(X)
u_old, p_old = split(x_old)

Re = Constant(1)
f = Expression((
    '-2 * cos(x[0]) * sin(x[1]) * (cos(2 * t) + sin(2 * t) / Re)',
    '2 * sin(x[0]) * cos(x[1]) * (cos(2 * t) + sin(2 * t) / Re)'
), degree=degree+1, t=0.0, Re=Re)
dt = 1.e-1

# backward Euler explicit scheme
#a = (dot(u, v) / Constant(dt) + inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
#L = (dot(u_old, v) / Constant(dt) - dot(grad(u_old) * u_old, v) + dot(f, v)) * dx

# backward Euler semi-implicit scheme
a = (dot(u, v) / Constant(dt) + dot(grad(u) * u_old, v) + inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
L = (dot(u_old, v) / Constant(dt) + dot(f, v)) * dx

# 4. solution
x = Function(X)
x_exact = Expression(('u[0]', 'u[1]', 'p'),
                     degree=degree+1, u=u_exact, p=p_exact)
x.interpolate(x_exact)

time = 0.0
max_eH1 = 0.0
max_eL2 = 0.0

while time < 1.0:
  time += dt
  f.t = time
  u_exact.t = time
  p_exact.t = time

  x_old.assign(x)
  solve(a == L, x, bc)
  
  u, p = x.split()
  eH1 = errornorm(u_exact, u, 'H1')
  eL2 = errornorm(p_exact, p, 'L2')
  print('time={:.2} eH1={:.3e} eL2={:.3e}'.format(time, eH1, eL2))
  
  max_eH1 = max(max_eH1, eH1)
  max_eL2 = max(max_eL2, eL2)
  
u, p = x.split()

u.rename('velocity', 'velocity')
p.rename('pressure', 'pressure')

print('error eH1={:.3e} eL3={:.3e}'.format(max_eH1, max_eL2))


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FF